<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Queries-from-Different-Languages" data-toc-modified-id="Queries-from-Different-Languages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Queries from Different Languages</a></span></li><li><span><a href="#Django-Interface" data-toc-modified-id="Django-Interface-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Django Interface</a></span><ul class="toc-item"><li><span><a href="#Simple-Calls-from-Esper-Demos" data-toc-modified-id="Simple-Calls-from-Esper-Demos-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Simple Calls from Esper Demos</a></span></li></ul></li><li><span><a href="#Spark-Interface" data-toc-modified-id="Spark-Interface-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Spark Interface</a></span><ul class="toc-item"><li><span><a href="#Simple-Calls" data-toc-modified-id="Simple-Calls-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Simple Calls</a></span></li></ul></li><li><span><a href="#BigQuery/SQL" data-toc-modified-id="BigQuery/SQL-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>BigQuery/SQL</a></span><ul class="toc-item"><li><span><a href="#Simple-Calls" data-toc-modified-id="Simple-Calls-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Simple Calls</a></span></li></ul></li></ul></div>

# Queries from Different Languages

This document contains a few different ways to make queries and display them in the Esper interface through the Jupyter widget.

Right now, there are three ways to query the database of objects (which contains faces, bounding boxes, etc):
* Django ORM
* Spark SQL
* Google BigQuery

# Django Interface

## Simple Calls from Esper Demos

In [ ]:
# Show me all faces

from query.models import Face
from esper.stdlib import qs_to_result
from esper.prelude import esper_widget

stride = 1

esper_widget(qs_to_result(Face.objects.all(), stride=stride))

In [ ]:
# Show me a specific person

from query.models import FaceIdentity
from esper.stdlib import qs_to_result

face_identities = FaceIdentity.objects
print(face_identities.all().values()[0])

person_name="hillary clinton"

esper_widget(qs_to_result(face_identities.filter(identity__name=person_name, probability__gte=0.99)))

In [ ]:
# Show me a panel, the easy way

from esper.queries import panels
from esper.stdlib import simple_result, bbox_to_dict

esper_widget(simple_result([
    {
        'video': frame.video.id,
        'min_frame': frame.number,
        'objects': [bbox_to_dict(f) for f in faces]
    } for (frame, faces) in panels()
], 'Panel'))

In [ ]:
# Show me a panel, the hard way (do it all yourself)

from query.models import Face, Frame
from django.db.models import OuterRef, IntegerField
from esper.stdlib import simple_result, bbox_to_dict
from esper.prelude import esper_widget

all_faces = Face.objects

# First, make sure all bounding boxes are big enough
faces_with_height = Face.objects.annotate(height=F('bbox_y2')-F('bbox_y1'))
faces_tall_enough = faces_with_height.filter(height__gte=.25)

# Next, annotate all frames with number of faces that appear in frame
frames_with_face_counts = Frame.objects.annotate(c=Subquery(
    faces_tall_enough.filter(person__frame=OuterRef('pk')) # Query for faces that are in this frame
    .values('person__frame') # Select the frame number, i.e. SELECT frame FROM ...
    .annotate(c=Count('*')) # Count, i.e. SELECT count(frame) FROM ...
    .values('c'), IntegerField() # SELECT count(frame) AS c FROM ...
))

# Next, filter so there are only N faces in each frame
N = 3
frames_with_N_faces = frames_with_face_counts.filter(c=N)

# Manually process 10000 frames and make sure that the y1's of the faces are at the same height
results = []
for frame in frames_with_N_faces[:10000:10]:
    faces = faces_tall_enough.filter(person__frame=frame)
    if len(faces) != 3:
        continue
    y1 = faces[0].bbox_y1
    y2 = faces[0].bbox_y2
    if (abs(faces[1].bbox_y1 - y1) > 0.05 or abs(faces[2].bbox_y1 - y1) > 0.05
        or abs(faces[1].bbox_y2 - y2) > 0.05 or abs(faces[2].bbox_y2 - y2) > 0.05):
        continue
    results.append({
        'video': frame.video.id,
        'min_frame': frame.number,
        'objects': [bbox_to_dict(f) for f in faces]
    })

# Finally, show the results
esper_widget(simple_result(results, 'Panel'))

# Spark Interface

NOTE: It appears that there can only be one connection to Spark through `SparkWrapper` at any one time (across all notebooks). If you want to start up a Spark connection on another notebook, you need to restart the Docker container.

## Simple Calls

In [ ]:
# Show me all faces, method 1

from esper.spark import SparkWrapper
from esper.stdlib import simple_result, bbox_to_dict
from esper.prelude import esper_widget, collect
from operator import itemgetter

spark = SparkWrapper()

faces = spark.load('query_face')
people = spark.load('query_person')
frames = spark.load('query_frame')

def foreign_key_join(df1, keys_and_tables):
    df = df1
    for (key, table) in keys_and_tables:
        df = df.join(table, df[key] == table.id).drop(table.id)
    return df

stride = 1000
limit = 100

faces_with_frames = foreign_key_join(faces, [('person_id', people),
                                            ('frame_id', frames)]) \
                                    .select('id', 'frame_id', 'video_id', 'number',
                                           'bbox_x1', 'bbox_x2', 'bbox_y1', 'bbox_y2') \
                                    .orderBy('video_id', 'number') \
                                    .withColumn('m', people.frame_id % stride) \
                                    .filter('m=0') \
                                    .limit(100).collect()

frames_used = set()
for frame in faces_with_frames:
    frames_used.add((frame.video_id, frame.number, frame.frame_id))
frames_used = list(frames_used)
frames_used.sort(key=itemgetter(0, 1))

relevant_faces = collect(
    faces_with_frames,
    lambda face : face.frame_id
)

materialized_results = []
for (video, number, frame_id) in frames_used:
    materialized_results.append({
        'video': video,
        'min_frame': number,
        'objects': [bbox_to_dict(face) for face in relevant_faces[frame_id]]
    })

esper_widget(simple_result(materialized_results, 'Faces'))

In [ ]:
# Show me all faces, method 2

from esper.spark import SparkWrapper
from esper.stdlib import simple_result, bbox_to_dict
from esper.prelude import esper_widget
from pyspark.sql.functions import collect_list

spark = SparkWrapper()

faces = spark.load('query_face')
people = spark.load('query_person')
frames = spark.load('query_frame')

def foreign_key_join(df1, keys_and_tables):
    df = df1
    for (key, table) in keys_and_tables:
        df = df.join(table, df[key] == table.id).drop(table.id)
    return df

stride = 1000
limit = 100

faces_with_frames = foreign_key_join(faces, [('person_id', people),
                                            ('frame_id', frames)]) \
                                    .select('id', 'frame_id', 'video_id', 'number',
                                           'bbox_x1', 'bbox_x2', 'bbox_y1', 'bbox_y2') \
                                    .orderBy('video_id', 'number') \
                                    .withColumn('m', people.frame_id % stride) \
                                    .filter('m=0') \
                                    .drop('m') \
                                    .groupBy('video_id', 'frame_id', 'number').agg(
                                        collect_list('id'),
                                        collect_list('bbox_x1'),
                                        collect_list('bbox_x2'),
                                        collect_list('bbox_y1'),
                                        collect_list('bbox_y2')
                                    )

materialized_results = []
for frame in faces_with_frames.collect():
    bboxes = [{
        'id': frame['collect_list(id)'][i],
        'bbox_x1': frame['collect_list(bbox_x1)'][i],
        'bbox_x2': frame['collect_list(bbox_x2)'][i],
        'bbox_y1': frame['collect_list(bbox_y1)'][i],
        'bbox_y2': frame['collect_list(bbox_y2)'][i],
        'type': 'bbox'
    } for i in range(len(frame['collect_list(id)']))]
    
    materialized_results.append({
        'video': frame.video_id,
        'min_frame': frame.number,
        'objects': bboxes
    })

esper_widget(simple_result(materialized_results, 'Faces'))

In [ ]:
# Helper functions

from operator import itemgetter
from esper.prelude import collect
from esper.stdlib import simple_result

def foreign_key_join(df1, keys_and_tables):
    df = df1
    for (key, table) in keys_and_tables:
        df = df.join(table, df[key] == table.id).drop(table.id)
    return df

def faces_to_result(faces, people, frames, stride, limit):
    faces_with_frames = foreign_key_join(faces, [('person_id', people),
                                                ('frame_id', frames)]) \
                                        .select('id', 'frame_id', 'video_id', 'number',
                                               'bbox_x1', 'bbox_x2', 'bbox_y1', 'bbox_y2') \
                                        .orderBy('video_id', 'number') \
                                        .withColumn('m', people.frame_id % stride) \
                                        .filter('m=0') \
                                        .limit(limit).collect()

    frames_used = set()
    for frame in faces_with_frames:
        frames_used.add((frame.video_id, frame.number, frame.frame_id))
    frames_used = list(frames_used)
    frames_used.sort(key=itemgetter(0, 1))

    relevant_faces = collect(
        faces_with_frames,
        lambda face : face.frame_id
    )

    materialized_results = []
    for (video, number, frame_id) in frames_used:
        materialized_results.append({
            'video': video,
            'min_frame': number,
            'objects': [bbox_to_dict(face) for face in relevant_faces[frame_id]]
        })
    
    return simple_result(materialized_results, 'Faces')

In [ ]:
# Get me all the faces of a single person

from esper.spark import SparkWrapper
from esper.prelude import esper_widget
from esper.stdlib import bbox_to_dict

spark = SparkWrapper()

faceidentities = spark.load('query_faceidentity')
identities = spark.load('query_identity')
faces = spark.load('query_face')
people = spark.load('query_person')
frames = spark.load('query_frame')

person_name = 'hillary clinton'

faces = foreign_key_join(faceidentities, [('identity_id', identities),
                                 ('face_id', faces)]) \
                            .filter(identities.name==person_name) \
                            .filter(faceidentities.probability >= 0.99) \
                            .select('id', 'person_id', 'bbox_x1', 'bbox_x2',
                                    'bbox_y1', 'bbox_y2')

materialized_faces = faces_to_result(faces, people, frames, 1000, 100)

esper_widget(materialized_faces)

In [ ]:
# Get me all panels
# NOTE: because of the way that the limiting works, some panels may be incomplete
#   (limiting faces instead of frames)

from esper.spark import SparkWrapper
from esper.prelude import esper_widget
from pyspark.sql import functions as F

spark = SparkWrapper()

faces = spark.load('query_face').filter('bbox_y2-bbox_y1 > 0.25')
people = spark.load('query_person')
frames = spark.load('query_frame')

expr = [F.count(F.col('*')), F.max(F.col('bbox_y1')), F.min(F.col('bbox_y1')),
       F.max(F.col('bbox_y2')), F.min(F.col('bbox_y2'))]

frames_with_three_faces = foreign_key_join(faces, [('person_id', people),
                                                ('frame_id', frames)]) \
                                        .groupBy('frame_id', 'video_id', 'number') \
                                        .agg(*expr) \
                                        .filter(F.col('count(1)') == 3) \
                                        .filter(F.col('max(bbox_y1)') - F.col('min(bbox_y1)') < .05) \
                                        .filter(F.col('max(bbox_y2)') - F.col('min(bbox_y2)') < .05) \
                                        .drop('count(1)', 'max(bbox_y1)', 'min(bbox_y1)',
                                             'max(bbox_y2)', 'min(bbox_y2)') \
                                        .toDF('id', 'video_id', 'number')

esper_widget(faces_to_result(faces, people, frames_with_three_faces, 1000, 300))

# BigQuery/SQL

BigQuery queries are run through the online web portal; it is also possible to run them through the command line. There may also be other ways to run these queries, and they should be runnable via Spark as well.

## Simple Calls

Faces, Panels, etc.

Faces: https://console.cloud.google.com/bigquery?sq=50518136478:74510b4bb1f04cc691fb470e5cf2fd86

```SQL
# Query to get all faces, with enough info to materialize

SELECT
  face.id, face.bbox_x1, face.bbox_x2, face.bbox_y1, face.bbox_y2,
  face.person_id, person.frame_id, frame.video_id, frame.number
FROM
  tvnews.query_face AS face,
  tvnews.query_person AS person,
  tvnews.query_frame AS frame
WHERE
  face.person_id = person.id
  AND person.frame_id = frame.id
LIMIT 100;
```

A specific face (Hillary Clinton): https://console.cloud.google.com/bigquery?sq=50518136478:d5257dee39b9413e9eeff5c68b42b038

```SQL
# Query to get all faces that belong to a particular person, with enough info to materialize

SELECT DISTINCT
  face.id, face.bbox_x1, face.bbox_x2, face.bbox_y1, face.bbox_y2,
  face.person_id, person.frame_id, frame.video_id, frame.number, identity.name
FROM
  tvnews.query_faceidentity AS faceidentity,
  tvnews.query_identity AS identity,
  tvnews.query_face AS face,
  tvnews.query_person AS person,
  tvnews.query_frame AS frame
WHERE
  identity.name = 'hillary clinton'
  AND faceidentity.face_id = face.id
  AND faceidentity.identity_id = identity.id
  AND faceidentity.probability > 0.99
  AND face.person_id = person.id
  AND person.frame_id = frame.id
LIMIT 100;
```

Panels: https://console.cloud.google.com/bigquery?sq=50518136478:a56f33c482cf4d94b6d4f54c053901c4

```SQL
# Get me all panels

SELECT
  face.id, frame_id, video_id, number,
  bbox_x1, bbox_x2, bbox_y1, bbox_y2
FROM
  tvnews.query_face AS face,
  tvnews.query_person AS person,
  tvnews.query_frame AS frame
WHERE
  frame_id IN (
    SELECT
      frame_id
    FROM
      tvnews.query_face AS face,
      tvnews.query_person AS person,
      tvnews.query_frame AS frame
    WHERE 
      bbox_y2 - bbox_y1 > 0.25
      AND face.person_id = person.id
      AND person.frame_id = frame.id
    GROUP BY frame_id
    HAVING
      COUNT(*) = 3
      AND MAX(bbox_y1) - MIN(bbox_y1) < .05
      AND MAX(bbox_y2) - MIN(bbox_y2) < .05
  )
  AND face.person_id = person.id
  AND person.frame_id = frame.id
LIMIT 100;
```